In [10]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import glob
from collections import defaultdict

## Core components

#### Parse JSON logs

In [56]:
def parse_champions(logs_file_iter):
    result = {}
    current_line = next(logs_file_iter, '{}')
    line_json = json.loads(current_line)
    while line_json.get('type', None) == 'ChampionSpawnedReport':
        name = line_json['value']['controller_name']
        coords = line_json['value']['coords']
        champion_data = {
            'name': name,
            'start_coords': coords,
            'last_coords': coords,
            'failed_steps': [],  # every item is a dict {'episode', 'error_text', 'coords'}
            'actions': [],
            'episode_died': -1,
            'last_attack_type': None
        }
        result[name] = champion_data
        current_line = next(logs_file_iter, '{}')
        line_json = json.loads(current_line)
    return result, current_line if current_line != '{}' else None

In [60]:
def parse_game_course(current_line, logs_file_iter, game_report):
    if current_line is None:
        return None
    champions_info = game_report['bots']
    
    line_json = json.loads(current_line)
    episode_number = 0
    mist_started = False
    while line_json.get('type', None) not in ('GameStartReport', None):
        event_type = line_json['type']
        bot_name = line_json.get('value', {}).get('controller_name', None)
        bot_data = champions_info.get(bot_name, None)
        
        if event_type == 'EpisodeStartReport':
            game_report['episodes_number'] = episode_number
            episode_number = int(line_json['value']['episode_number'])
        elif line_json['severity'] == 'WARNING':
            error_text = line_json['value']['exception']
            coords = bot_data['last_coords']
            bot_data['failed_steps'].append({
                'episode': episode_number,
                'error_text': error_text,
                'coords': coords
            })
        elif event_type == 'ChampionPickedActionReport':
            action = line_json['value']['action_name']
            bot_data['actions'].append(action)
        elif event_type == 'ChampionEnteredTileReport':
            new_coords = line_json['value']['tile_coords']
            bot_data['last_coords'] = new_coords
        elif event_type == 'ChampionDeathReport':
            bot_data['episode_died'] = episode_number
        elif event_type == 'ChampionDamagedByMistReport':
            bot_data['last_attack_type'] = 'mist'
        elif event_type == 'ChampionDamagedByWeaponCutReport':
            bot_data['last_attack_type'] = 'weapon'
        elif event_type == 'MistRadiusReducedReport' and not mist_started:
            mist_started = True
            game_report['mist_episode'] = episode_number
        
        current_line = next(logs_file_iter, '{}')
        line_json = json.loads(current_line)
    return current_line if current_line != '{}' else None

In [61]:
def parse_games_json_logs(logs_path):
    with open(logs_path) as logs_file:
        logs_file_iter = iter(logs_file)
        result = []
        current_line = next(logs_file_iter, None)
        while current_line is not None:
            game_json = json.loads(current_line)
            arena_json = json.loads(next(logs_file_iter, 'null'))
            menhir_json = json.loads(next(logs_file_iter, 'null'))
            champions, current_line = parse_champions(logs_file_iter)
            if not all((arena_json, menhir_json, champions)):
                print('Not all metadata provided')
                break
            game_report = {
                'game_number': int(game_json['value']['game_number']),
                'mist_episode': -1,
                'menhir_pos': menhir_json['value']['position'],
                'arena_name': arena_json['value']['arena_name'],
                'bots': champions,
                'episodes_number': 0
            }

            current_line = parse_game_course(current_line, logs_file_iter, game_report)
            result.append(game_report)
    return result

#### Parsed data analysis

## Before running the parser
> The parser `.ipynb` is put under `GUPB/log-parse` and all paths to logs are relative to this location. Be sure that the files are downloaded and placed there, because they are not under Git

## Example log

In [62]:
example_path = 'example_log.json'
example_parsed_data = parse_games_json_logs(example_path)

In [66]:
[f'{key}: {example_parsed_data[0][key]}' for key in example_parsed_data[0] if key != 'bots']

['game_number: 1',
 'mist_episode: 25',
 'menhir_pos: [9, 9]',
 'arena_name: isolated_shrine',
 'episodes_number: 477']

## 11:15 games

In [9]:
DIR_PATH_11_15 = '11_15__2021_10_24'
log_files = glob.glob(f'{DIR_PATH_11_15}/*.json')

In [21]:
json_log_path = log_files[0]

In [ ]:
games = 